In [2]:
import sys
sys.path.append('../')
sys.path.append('/Users/sdbykov/work/my_packages/nway')

In [3]:
from scripts.utils import data_path, set_mpl
from scripts.cross_match_scripts import cat2hpx, pandas_to_fits, my_scaler_forward, desi_reliable_magnitudes
from tqdm import tqdm
import pandas as pd
import numpy as np
import seaborn as sns
import astropy.io.fits as fits
import healpy as hp
from tensorflow import keras

import nway_python_api as nway
%matplotlib inline

This is NWAY Python API
nwaylib file /Users/sdbykov/work/my_packages/nway/nwaylib/__init__.py


# LH DESI catalog

All DESI objects within LH area + a little boundary


In [4]:
desi_cat_orig = pd.read_pickle(data_path+'desi_lh.gz_pkl', compression='gzip')

In [5]:
desi_cat_orig = desi_reliable_magnitudes(desi_cat_orig, s_n_threshold=3) #make cuts for SNR of flux + add colors
desi_cat_orig

/Users/sdbykov/opt/miniconda3/envs/venv_hea/lib/python3.10/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/sdbykov/opt/miniconda3/envs/venv_hea/lib/python3.10/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/sdbykov/opt/miniconda3/envs/venv_hea/lib/python3.10/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/sdbykov/opt/miniconda3/envs/venv_hea/lib/python3.10/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/sdbykov/opt/miniconda3/envs/venv_hea/lib/python3.10/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: invalid value encountered in log10
  result = getat

,release,objid,brickid,ra,dec,flux_g,flux_r,flux_z,flux_w1,flux_w2,flux_w3,flux_w4,flux_ivar_g,flux_ivar_r,flux_ivar_z,flux_ivar_w1,flux_ivar_w2,flux_ivar_w3,flux_ivar_w4,dered_mag_g,dered_mag_r,dered_mag_z,dered_mag_w1,dered_mag_w2,dered_mag_w3,dered_mag_w4,snr_g,snr_r,snr_z,snr_w1,snr_w2,snr_w3,snr_w4,parallax,parallax_ivar,pmra,pmra_ivar,pmdec,pmdec_ivar,ref_cat,ref_id,desi_id,all_mag_g,rel_mag_g,rel_dered_mag_g,all_mag_r,rel_mag_r,rel_dered_mag_r,all_mag_z,rel_mag_z,rel_dered_mag_z,all_mag_w1,rel_mag_w1,rel_dered_mag_w1,vega_mag_w1,all_mag_w2,rel_mag_w2,rel_dered_mag_w2,vega_mag_w2,all_mag_w3,rel_mag_w3,rel_dered_mag_w3,vega_mag_w3,all_mag_w4,rel_mag_w4,rel_dered_mag_w4,vega_mag_w4,rel_desi_flux_corr_z,rel_dered_g_r,rel_dered_g_z,rel_dered_r_z,rel_dered_z_w1,rel_dered_r_w2,rel_dered_w1_w2,rel_dered_z_w3,rel_dered_r_w4,rel_dered_w3_w4
0,9011,256,623751,161.871392,62.272114,0.116844,0.419296,1.758394,2.764221,0.199589,30.088644,150.333180,649.10230,244.88066,41.276090,4.586966,1.156097,0.001470,0.000018,24.800236,23.422989,21.875626,21.394308,24.248579,18.803762,17.057276,2.976899,6.561414,11.297059,5.920189,0.214602,1.153541,0.629635,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0,9011_623751_256,24.830984,NaN,NaN,23.443698,23.443698,23.422989,21.887210,21.887210,21.875626,21.396068,21.396068,21.394308,18.697068,24.249658,NaN,NaN,NaN,18.803993,NaN,NaN,NaN,17.057363,NaN,NaN,NaN,2.967130e-15,NaN,NaN,1.547363,0.481318,NaN,NaN,NaN,NaN,NaN
1,9011,3023,623750,161.795663,62.309075,0.278886,0.729804,0.925065,-0.124014,0.842693,7.723400,-409.541560,628.14530,241.19875,67.662970,4.827379,1.161493,0.001375,0.000017,23.855670,22.821260,22.572979,NaN,22.684746,20.280249,NaN,6.989662,11.334282,7.609353,-0.272475,0.908192,0.286416,-1.682871,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0,9011_623750_3023,23.886433,23.886433,23.855670,22.841984,22.841984,22.821260,22.584569,22.584569,22.572979,NaN,NaN,NaN,NaN,22.685827,NaN,NaN,NaN,20.280479,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.560973e-15,1.034410,1.282691,0.248281,NaN,NaN,NaN,NaN,NaN,NaN
2,9011,2993,623750,161.789851,62.306325,0.832196,1.570136,3.506991,17.061989,15.794795,-11.218180,293.149100,381.48358,190.84026,46.138702,3.873229,1.025061,0.001371,0.000017,22.668740,21.989480,21.126097,19.418169,19.502636,NaN,16.332191,16.254122,21.690641,23.821402,33.578884,15.991490,-0.415374,1.201052,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0,9011_623750_2993,22.699436,22.699436,22.668740,22.010157,22.010157,21.989480,21.137663,21.137663,21.126097,19.419926,19.419926,19.418169,16.720926,19.503715,19.503715,19.502636,16.164715,NaN,NaN,NaN,NaN,16.332279,NaN,NaN,NaN,5.917634e-15,0.679260,1.542643,0.863383,1.707928,2.486844,-0.084467,NaN,NaN,NaN
3,9011,3026,623750,161.796264,62.305792,0.264495,0.484687,0.485263,-0.030457,-0.994314,-2.437728,215.115220,423.51940,211.87927,52.782760,4.773809,1.144707,0.001378,0.000017,23.913218,23.265640,23.273476,NaN,NaN,NaN,16.668236,5.443207,7.055143,3.525519,-0.066547,-1.063826,-0.090477,0.881277,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0,9011_623750_3026,23.943956,23.943956,23.913218,23.286347,23.286347,23.265640,23.285057,23.285057,23.273476,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.668322,NaN,NaN,NaN,8.188353e-16,0.647578,0.639742,-0.007836,NaN,NaN,NaN,NaN,NaN,NaN
4,9011,3020,623750,161.794747,62.300601,0.304706,0.452320,1.219659,1.665158,0.451321,10.007744,-11.781795,650.39940,257.07184,72.178210,4.741981,1.172464,0.001389,0.000017,23.759617,23.340720,22.272844,21.944605,23.362709,19.998930,NaN,7.770905,7.252247,10.361954,3.626062,0.488692,0.372935,-0.048081,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0,9011_623750_3020,23.790297,23.790297,23.759617,23.361386,23.361386,23.340720,22.284404,22.284404,22.272844,21.946361,21.946361,21.944605,19.247361,23.363786,NaN,NaN,NaN,19.999160,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.058019e-15,0.418897,1.486773,1.067876,0.328239,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

countring missing data

In [6]:
print('grz is not nan:', len(desi_cat_orig.query('~rel_dered_mag_g.isna() & ~rel_dered_mag_r.isna() & ~rel_dered_mag_z.isna()'))/len(desi_cat_orig))

print('grz is not nan, w1 is not nan, w2 is nan:', len(desi_cat_orig.query('~rel_dered_mag_g.isna() & ~rel_dered_mag_r.isna() & ~rel_dered_mag_z.isna() & ~rel_dered_mag_w1.isna() & rel_dered_mag_w2.isna()'))/len(desi_cat_orig))

print('grz is not nan, w2 is not nan, w1 is nan:', len(desi_cat_orig.query('~rel_dered_mag_g.isna() & ~rel_dered_mag_r.isna() & ~rel_dered_mag_z.isna() & rel_dered_mag_w1.isna() & ~rel_dered_mag_w2.isna()'))/len(desi_cat_orig))


print('grz is not nan, w1 and w2 is not nan', len(desi_cat_orig.query('~rel_dered_mag_g.isna() & ~rel_dered_mag_r.isna() & ~rel_dered_mag_z.isna() & ~rel_dered_mag_w1.isna() & ~rel_dered_mag_w2.isna()'))/len(desi_cat_orig))

grz is not nan: 0.7246063176069866
grz is not nan, w1 is not nan, w2 is nan: 0.19196973092408998
grz is not nan, w2 is not nan, w1 is nan: 0.008405366137236239
grz is not nan, w1 and w2 is not nan 0.29915355081134587


filtering the catalog

In [7]:
desi_cat = desi_cat_orig.sort_values(by=['flux_g'], ascending=[False])
desi_cat = desi_cat.drop_duplicates(subset=['ra', 'dec'], keep='first')



columns_to_retain = ['desi_id', 'ra', 'dec', 'rel_dered_mag_g','rel_dered_mag_r','rel_dered_mag_z','rel_dered_mag_w1','rel_dered_mag_w2', 'rel_dered_mag_w3', 'rel_dered_mag_w4', 'rel_dered_g_r', 'rel_dered_r_z', 'rel_dered_g_z','rel_dered_z_w1', 'rel_dered_r_w2', 'rel_dered_w1_w2', 'rel_dered_z_w3', 'rel_dered_r_w4', 'rel_dered_w3_w4']

desi_cat = desi_cat[columns_to_retain]


desi_cat.reset_index(inplace=True, drop = True)
desi_cat.sample(5)

,desi_id,ra,dec,rel_dered_mag_g,rel_dered_mag_r,rel_dered_mag_z,rel_dered_mag_w1,rel_dered_mag_w2,rel_dered_mag_w3,rel_dered_mag_w4,rel_dered_g_r,rel_dered_r_z,rel_dered_g_z,rel_dered_z_w1,rel_dered_r_w2,rel_dered_w1_w2,rel_dered_z_w3,rel_dered_r_w4,rel_dered_w3_w4
1098790,9011_605191_2181,163.755966,56.123364,23.674246,22.893826,22.262194,21.959180,NaN,NaN,NaN,0.780420,0.631632,1.412052,0.303014,NaN,NaN,NaN,NaN,NaN
1022958,9011_600233_3096,159.292519,54.448343,23.589054,23.466390,23.138943,NaN,NaN,NaN,NaN,0.122664,0.327447,0.450111,NaN,NaN,NaN,NaN,NaN,NaN
2058789,9011_608390_3877,165.809525,56.893976,NaN,23.107357,21.783573,22.051700,NaN,NaN,NaN,NaN,1.323784,NaN,-0.268127,NaN,NaN,NaN,NaN,NaN
71018,9011_615997_3790,161.849753,59.533530,20.046877,19.684158,19.694641,20.496643,NaN,NaN,NaN,0.362719,-0.010483,0.352236,-0.802002,NaN,NaN,NaN,NaN,NaN
2203593,9011_601907_3224,161.266165,55.026515,NaN,23.559246,23.150898,NaN,NaN,NaN,NaN,NaN,0.408348,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Dealing with missing data -> assigning prior type

In [8]:
desi_cat['prior_type'] = 'None'

mask = ~(desi_cat.rel_dered_mag_g.isna()) & ~(desi_cat.rel_dered_mag_r.isna()) & ~(desi_cat.rel_dered_mag_z.isna())
desi_cat.loc[mask, 'prior_type'] = 'grz'
mask = mask & ~(desi_cat.rel_dered_mag_w1.isna())
desi_cat.loc[mask, 'prior_type'] = 'grzw1'
mask = mask & ~(desi_cat.rel_dered_mag_w2.isna()) 
desi_cat.loc[mask, 'prior_type'] = 'grzw1w2'

#desi_cat_scaled_imputed.loc[desi_cat_scaled_imputed.prior_type==prior, features_cols] = -99



In [9]:
print(desi_cat.prior_type.value_counts())
print(desi_cat.prior_type.value_counts(normalize=1))

grzw1w2    723525
None       666060
grz        564696
grzw1      464293
Name: prior_type, dtype: int64
grzw1w2    0.299154
None       0.275394
grz        0.233483
grzw1      0.191970
Name: prior_type, dtype: float64


## Adding prior columns

In [10]:
desi_cat_scaled = my_scaler_forward(desi_cat) #scale the data for KERAS NN applying

rel_dered_mag_g scaled by 1/35
rel_dered_mag_r scaled by 1/35
rel_dered_mag_z scaled by 1/35
rel_dered_mag_w1 scaled by 1/35
rel_dered_mag_w2 scaled by 1/35
rel_dered_mag_w3 scaled by 1/35
rel_dered_mag_w4 scaled by 1/35
rel_dered_g_r scaled by 1/10
rel_dered_r_z scaled by 1/10
rel_dered_g_z scaled by 1/10
rel_dered_z_w1 scaled by 1/10
rel_dered_r_w2 scaled by 1/10
rel_dered_w1_w2 scaled by 1/10
rel_dered_z_w3 scaled by 1/10
rel_dered_r_w4 scaled by 1/10
rel_dered_w3_w4 scaled by 1/10


In [11]:
#print(pd.DataFrame(desi_cat_scaled.describe().loc[['mean', 'std',  'max', 'min']]).T)
desi_cat_scaled_imputed= desi_cat_scaled.fillna(value = -99)


In [23]:
for prior in ['grzw1w2', 'grzw1', 'grz']:
    nnmag_name = 'nnmag_'+prior
    desi_cat_scaled_imputed[nnmag_name] = -99
    nnmag_name = 'nnmag_'+prior+'_orig'
    desi_cat_scaled_imputed[nnmag_name] = -99




In [26]:
model_grz = keras.models.load_model('/Users/sdbykov/work/lockman_hole/1_desi-photo-prior/model/nnmag_grz.keras_nn')
model_grzw1 = keras.models.load_model('/Users/sdbykov/work/lockman_hole/1_desi-photo-prior/model/nnmag_grzw1.keras_nn')
model_grzw1w2 = keras.models.load_model('/Users/sdbykov/work/lockman_hole/1_desi-photo-prior/model/nnmag_grzw1w2.keras_nn')


def make_predictions(df, prior_type):

    features_cols_grzw1w2 =  ['rel_dered_mag_g', 'rel_dered_mag_r', 'rel_dered_mag_z', 'rel_dered_mag_w1', 'rel_dered_mag_w2', 'rel_dered_g_r', 'rel_dered_r_z', 'rel_dered_g_z', 'rel_dered_z_w1', 'rel_dered_r_w2', 'rel_dered_w1_w2']
    features_cols_grzw1 =  ['rel_dered_mag_g', 'rel_dered_mag_r', 'rel_dered_mag_z', 'rel_dered_mag_w1', 'rel_dered_g_r', 'rel_dered_r_z', 'rel_dered_g_z', 'rel_dered_z_w1']
    features_cols_grz =  ['rel_dered_mag_g', 'rel_dered_mag_r', 'rel_dered_mag_z', 'rel_dered_g_r', 'rel_dered_r_z', 'rel_dered_g_z']

    print('Calculating predictions for prior_type: '+prior_type)
    if prior_type == 'grzw1w2':
        features_cols = features_cols_grzw1w2
        model = model_grzw1w2
    elif prior_type == 'grzw1':
        features_cols = features_cols_grzw1
        model = model_grzw1
    elif prior_type == 'grz':
        features_cols = features_cols_grz
        model = model_grz
    else:
        raise ValueError('prior_type type not recognized')

        
    mask = df.prior_type==prior_type
    df_tmp = df[mask][features_cols]


    nnmag_name = 'nnmag_'+prior_type
    X_predict = df_tmp.values
    y_predict = model.predict(X_predict)

    df.loc[mask, nnmag_name] = y_predict
    df.loc[mask, nnmag_name+'_orig'] = y_predict

    if prior_type=='grzw1' or prior_type=='grzw1w2':
        print('--Calculating sub-predictions for prior_type: '+'grz')

        X_predict =  df[mask][features_cols_grz].values
        y_predict = model_grz.predict(X_predict)
        df.loc[mask, 'nnmag_grz_orig'] = y_predict


    if prior_type=='grzw1w2':
        print('--Calculating sub-predictions for prior_type: '+'grzw1')
        X_predict =  df[mask][features_cols_grzw1].values
        y_predict = model_grzw1.predict(X_predict)
        df.loc[mask, 'nnmag_grzw1_orig'] = y_predict

    return df

for prior_type in ['grzw1w2', 'grzw1', 'grz']:
    desi_cat_scaled_imputed = make_predictions(desi_cat_scaled_imputed, prior_type)
   

Calculating predictions for prior_type: grzw1w2
22611/22611 [==============================] - 20s 870us/step
--Calculating sub-predictions for prior_type: grz
22611/22611 [==============================] - 26s 1ms/step
--Calculating sub-predictions for prior_type: grzw1
22611/22611 [==============================] - 20s 877us/step
Calculating predictions for prior_type: grzw1
14510/14510 [==============================] - 12s 847us/step
--Calculating sub-predictions for prior_type: grz
14510/14510 [==============================] - 19s 1ms/step
Calculating predictions for prior_type: grz
17647/17647 [==============================] - 15s 839us/step


In [27]:
desi_cat_scaled_imputed.query("nnmag_grz_orig!=-99")

,desi_id,ra,dec,rel_dered_mag_g,rel_dered_mag_r,rel_dered_mag_z,rel_dered_mag_w1,rel_dered_mag_w2,rel_dered_mag_w3,rel_dered_mag_w4,rel_dered_g_r,rel_dered_r_z,rel_dered_g_z,rel_dered_z_w1,rel_dered_r_w2,rel_dered_w1_w2,rel_dered_z_w3,rel_dered_r_w4,rel_dered_w3_w4,prior_type,nnmag_grzw1w2,nnmag_grzw1w2_orig,nnmag_grzw1,nnmag_grzw1_orig,nnmag_grz,nnmag_grz_orig
20,9011_608391_3304,166.193423,56.994008,0.235522,0.174716,0.417170,0.348593,0.367892,0.405444,0.416527,0.212819,-0.848586,-0.635767,0.240018,-0.676117,-0.067548,0.041039,-0.846337,-0.038790,grzw1w2,0.363125,0.363125,-99.000000,0.763354,-99.000000,0.204797
31,9011_611477_2811,155.048961,57.897765,0.248089,0.370034,0.388594,0.322769,0.341074,0.390173,0.416863,-0.426805,-0.064961,-0.491766,0.230386,0.101359,-0.064066,-0.005527,-0.163904,-0.093417,grzw1w2,0.946184,0.946184,-99.000000,0.933835,-99.000000,0.997413
35,9011_605987_1393,159.127645,56.156413,0.253675,0.288068,0.349741,0.322860,0.342291,0.390584,0.413188,-0.120375,-0.215855,-0.336230,0.094082,-0.189779,-0.068006,-0.142950,-0.437920,-0.079115,grzw1w2,0.876454,0.876454,-99.000000,0.933833,-99.000000,0.676145
39,9011_603553_3101,159.774676,55.512338,0.259401,0.323637,0.426948,0.283841,0.295961,0.347407,0.381680,-0.224824,-0.361589,-0.586413,0.500874,0.096866,-0.042419,0.278393,-0.203152,-0.119957,grzw1w2,0.848700,0.848700,-99.000000,0.933826,-99.000000,0.098723
46,9011_608371_1748,156.891014,56.878860,0.264951,0.322180,0.427598,0.305930,0.323325,0.373075,0.409076,-0.200300,-0.368964,-0.569264,0.425838,-0.004009,-0.060883,0.190830,-0.304138,-0.126003,grzw1w2,0.849171,0.849171,-99.000000,0.933829,-99.000000,0.101344
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2166732,9011_602739_3367,163.273652,55.206833,0.718991,0.674090,0.662804,-99.000000,-99.000000,-99.000000,-99.000000,0.157155,0.039499,0.196654,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,grz,-99.000000,-99.000000,-99.000000,-99.000000,0.069951,0.069951
2170166,9011_602740_3460,163.722208,55.210597,0.719333,0.693232,0.642976,0.582317,0.592495,-99.000000,-99.000000,0.091354,0.175897,0.267250,0.212306,0.352580,-0.035623,-99.000000,-99.000000,-99.000000,grzw1w2,0.060661,0.060661,-99.000000,0.220174,-99.000000,0.314074
2173122,9011_602740_2087,163.599114,55.220587,0.719602,0.662111,0.620208,-99.000000,-99.000000,-99.000000,-99.000000,0.201220,0.146661,0.347880,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,grz,-99.000000,-99.000000,-99.000000,-99.000000,0.197002,0.197002
2181308,9011_601078_4008,162.729909,54.805880,0.720728,0.675197,0.633285,0.611307,0.608974,-99.000000,-99.000000,0.159358,0.146691,0.306049,0.076925,0.231779,0.008163,-99.000000,-99.000000,-99.000000,grzw1w2,0.043866,0.043866,-99.000000,0.020294,-99.000000,0.247011


In [14]:
#desi_cat_scaled_imputed_no_nnmag_grz = desi_cat_scaled_imputed.copy() # this is to test the case when i ignore nnmag-grz prior and left it as is, i.e. only grzw1w2(w3w4) trained priors are used. uncomment respective lines below to make a catalog without grz model


In [28]:
for prior in ['grzw1w2', 'grzw1', 'grz']:


    features_cols_grzw1w2 =  ['rel_dered_mag_g', 'rel_dered_mag_r', 'rel_dered_mag_z', 'rel_dered_mag_w1', 'rel_dered_mag_w2', 'rel_dered_g_r', 'rel_dered_r_z', 'rel_dered_g_z', 'rel_dered_z_w1', 'rel_dered_r_w2', 'rel_dered_w1_w2']
    features_cols_grzw1 =  ['rel_dered_mag_g', 'rel_dered_mag_r', 'rel_dered_mag_z', 'rel_dered_mag_w1', 'rel_dered_g_r', 'rel_dered_r_z', 'rel_dered_g_z', 'rel_dered_z_w1']
    features_cols_grz =  ['rel_dered_mag_g', 'rel_dered_mag_r', 'rel_dered_mag_z', 'rel_dered_g_r', 'rel_dered_r_z', 'rel_dered_g_z']

    print('Removing original  features columns for prior: '+prior)
    if prior == 'grzw1w2':
        features_cols = features_cols_grzw1w2
    elif prior == 'grzw1':
        features_cols = features_cols_grzw1
    elif prior == 'grz':
        features_cols = features_cols_grz
    else:
        raise ValueError('prior type not recognized')


    mask = desi_cat_scaled_imputed.prior_type==prior
    desi_cat_scaled_imputed.loc[mask, features_cols] = -99


Removing original  features columns for prior: grzw1w2
Removing original  features columns for prior: grzw1
Removing original  features columns for prior: grz


In [15]:

# for prior in ['grzw1w2w3w4', 'grzw1w2']:
#     if prior == 'grzw1w2':
#         features_cols = ['mag_g','mag_r','mag_z','mag_w1','mag_w2', 'col_gr', 'col_rz',  'col_gz','col_zw1', 'col_rw2', 'col_w1w2']
#     elif prior == 'grzw1w2w3w4':
#         features_cols = ['mag_g','mag_r','mag_z','mag_w1','mag_w2', 'mag_w3', 'mag_w4', 'col_gr', 'col_rz', 'col_gz','col_zw1', 'col_rw2', 'col_w1w2', 'col_zw3', 'col_rw4', 'col_w3w4']

#     mask = desi_cat_scaled_imputed_no_nnmag_grz.prior_type==prior
#     desi_cat_scaled_imputed_no_nnmag_grz.loc[mask, features_cols] = -99


In [29]:
photo_columns = ['desi_id', 'rel_dered_mag_g','rel_dered_mag_r','rel_dered_mag_z','rel_dered_mag_w1','rel_dered_mag_w2', 'rel_dered_mag_w3', 'rel_dered_mag_w4', 'rel_dered_g_r', 'rel_dered_r_z', 'rel_dered_g_z','rel_dered_z_w1', 'rel_dered_r_w2', 'rel_dered_w1_w2', 'rel_dered_z_w3', 'rel_dered_r_w4', 'rel_dered_w3_w4']

desi_cat_scaled_imputed = desi_cat_scaled_imputed.merge(desi_cat[photo_columns], on='desi_id', how='left', suffixes=('', '_orig'))

#desi_cat_scaled_imputed_no_nnmag_grz = desi_cat_scaled_imputed_no_nnmag_grz.merge(desi_cat[photo_columns], on='desi_id', how='left', suffixes=('', '_orig'))
desi_cat_scaled_imputed.sample(10)

,desi_id,ra,dec,rel_dered_mag_g,rel_dered_mag_r,rel_dered_mag_z,rel_dered_mag_w1,rel_dered_mag_w2,rel_dered_mag_w3,rel_dered_mag_w4,rel_dered_g_r,rel_dered_r_z,rel_dered_g_z,rel_dered_z_w1,rel_dered_r_w2,rel_dered_w1_w2,rel_dered_z_w3,rel_dered_r_w4,rel_dered_w3_w4,prior_type,nnmag_grzw1w2,nnmag_grzw1w2_orig,nnmag_grzw1,nnmag_grzw1_orig,nnmag_grz,nnmag_grz_orig,rel_dered_mag_g_orig,rel_dered_mag_r_orig,rel_dered_mag_z_orig,rel_dered_mag_w1_orig,rel_dered_mag_w2_orig,rel_dered_mag_w3_orig,rel_dered_mag_w4_orig,rel_dered_g_r_orig,rel_dered_r_z_orig,rel_dered_g_z_orig,rel_dered_z_w1_orig,rel_dered_r_w2_orig,rel_dered_w1_w2_orig,rel_dered_z_w3_orig,rel_dered_r_w4_orig,rel_dered_w3_w4_orig
1413502,9011_604362_335,155.712905,55.675631,-99.000000,-99.000000,-99.0000,-99.000000,-99.000000,-99.0,-99.0,-99.000000,-99.000000,-99.0,-99.000000,-99.000000,-99.000000,-99.0,-99.0,-99.0,grzw1,-99.000000,-99.000000,0.203848,0.203848,-99.000000,0.304164,23.961487,23.531113,22.273190,20.821240,NaN,NaN,NaN,0.430374,1.257923,1.688297,1.451950,NaN,NaN,NaN,NaN,NaN
146153,9011_622387_3713,160.570767,61.842954,-99.000000,-99.000000,-99.0000,-99.000000,-99.000000,-99.0,-99.0,-99.000000,-99.000000,-99.0,-99.000000,-99.000000,-99.000000,-99.0,-99.0,-99.0,grzw1,-99.000000,-99.000000,0.033797,0.033797,-99.000000,0.406338,21.199654,19.949171,19.195478,20.010084,NaN,NaN,NaN,1.250483,0.753693,2.004176,-0.814606,NaN,NaN,NaN,NaN,NaN
1848314,9011_605185_1392,160.992515,55.910355,-99.000000,0.660128,0.6253,0.575222,0.578589,-99.0,-99.0,-99.000000,0.121898,-99.0,0.175273,0.285384,-0.011786,-99.0,-99.0,-99.0,None,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,NaN,23.104475,21.885500,20.132774,20.250631,NaN,NaN,NaN,1.218975,NaN,1.752726,2.853844,-0.117857,NaN,NaN,NaN
1940576,9011_608387_3678,164.421563,57.026036,-99.000000,-99.000000,-99.0000,-99.000000,-99.000000,-99.0,-99.0,-99.000000,-99.000000,-99.0,-99.000000,-99.000000,-99.000000,-99.0,-99.0,-99.0,grz,-99.000000,-99.000000,-99.000000,-99.000000,0.238700,0.238700,24.571083,23.508274,23.825048,NaN,NaN,NaN,NaN,1.062809,-0.316774,0.746035,NaN,NaN,NaN,NaN,NaN,NaN
270058,9011_605985_2162,158.359702,56.316700,-99.000000,-99.000000,-99.0000,-99.000000,-99.000000,-99.0,-99.0,-99.000000,-99.000000,-99.0,-99.000000,-99.000000,-99.000000,-99.0,-99.0,-99.0,grzw1w2,0.013755,0.013755,-99.000000,0.046972,-99.000000,0.208522,22.061249,20.986322,20.531155,20.338130,21.321934,NaN,NaN,1.074927,0.455167,1.530094,0.193025,-0.335612,-0.983804,NaN,NaN,NaN
1639258,9011_621699_529,160.259740,61.533148,-99.000000,-99.000000,-99.0000,-99.000000,-99.000000,-99.0,-99.0,-99.000000,-99.000000,-99.0,-99.000000,-99.000000,-99.000000,-99.0,-99.0,-99.0,grzw1,-99.000000,-99.000000,0.087685,0.087685,-99.000000,0.173385,24.175032,23.707653,22.716722,21.801054,NaN,NaN,NaN,0.467379,0.990931,1.458310,0.915668,NaN,NaN,NaN,NaN,NaN
2007238,9011_613026_1523,166.851416,58.463221,-99.000000,-99.000000,-99.0000,-99.000000,-99.000000,-99.0,-99.0,-99.000000,-99.000000,-99.0,-99.000000,-99.000000,-99.000000,-99.0,-99.0,-99.0,grzw1,-99.000000,-99.000000,0.012632,0.012632,-99.000000,0.339324,24.708440,23.506240,22.111670,21.908806,NaN,NaN,NaN,1.202200,1.394570,2.596770,0.202864,NaN,NaN,NaN,NaN,NaN
346474,9011_603562_4223,163.731673,55.522868,-99.000000,-99.000000,-99.0000,-99.000000,-99.000000,-99.0,-99.0,-99.000000,-99.000000,-99.0,-99.000000,-99.000000,-99.000000,-99.0,-99.0,-99.0,grzw1,-99.000000,-99.000000,0.035812,0.035812,-99.000000,0.515232,22.377136,21.878570,21.650986,21.779533,NaN,NaN,NaN,0.498566,0.227584,0.726150,-0.128547,NaN,NaN,NaN,NaN,NaN
1600247,9011_612262_2173,164.663552,58.142107,0.689723,0.658308,-99.0000,-99.000000,-99.000000,-99.0,-99.0,0.109954,-99.000000,-99.0,-99.000000,-99.000000,-99.000000,-99.0,-99.0,-99.0,None,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,24.140310,23.040771,NaN,NaN,NaN,NaN,NaN,1.099539,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1316526,9011_611506_4044,168.720563,58.027868,-99.000000,-99.0

In [30]:
desi_cat_scaled_imputed.groupby('prior_type')[['nnmag_grzw1w2', 'nnmag_grzw1', 'nnmag_grz']].agg(['max'])


,nnmag_grzw1w2,nnmag_grzw1,nnmag_grz
,max,max,max
prior_type,,,
None,-99.00000,-99.000000,-99.000000
grz,-99.00000,-99.000000,0.999809
grzw1,-99.00000,0.933835,-99.000000
grzw1w2,0.97074,-99.000000,-99.000000


In [31]:
desi_cat_scaled_imputed.groupby('prior_type')[['nnmag_grzw1w2_orig', 'nnmag_grzw1_orig', 'nnmag_grz_orig']].agg(['max'])


,nnmag_grzw1w2_orig,nnmag_grzw1_orig,nnmag_grz_orig
,max,max,max
prior_type,,,
None,-99.00000,-99.000000,-99.000000
grz,-99.00000,-99.000000,0.999809
grzw1,-99.00000,0.933835,0.999851
grzw1w2,0.97074,0.933835,0.999880


## area and plotting

In [32]:
%matplotlib inline
#desi_hp_map = cat2hpx(desi_cat_scaled_imputed['ra'], desi_cat_scaled_imputed['dec'], nside=1024, radec=True)

try:
    hp.gnomview(desi_hp_map, rot = [150.23,52.19], xsize = 400)
    DESI_SKYAREA = 4*np.pi*(np.rad2deg(1))**2 * np.sum(desi_hp_map>=1)/len(desi_hp_map)
except:
    DESI_SKYAREA = 41.728630922842704
print(DESI_SKYAREA, 'sq. deg.')

desi_sky_dens = len(desi_cat_scaled_imputed)/DESI_SKYAREA
print('DESI sky density:', desi_sky_dens, 'per sq deg')
print('DESI sky density:', desi_sky_dens/(3600), 'per sq. arcmin')
print('DESI sky density:', desi_sky_dens/(3600**2), 'per sq. arcsec')
print(' expected number of DESI sources within 1 arcsec radius', np.pi*1**2 * desi_sky_dens/(3600**2))
print(' expected number of DESI sources within 5 arcsec radius', np.pi*5**2 * desi_sky_dens/(3600**2))
print(' expected number of DESI sources within 10 arcsec radius', np.pi*10**2 * desi_sky_dens/(3600**2))



41.728630922842704 sq. deg.
DESI sky density: 57959.58186291816 per sq deg
DESI sky density: 16.0998838508106 per sq. arcmin
DESI sky density: 0.0044721899585585 per sq. arcsec
 expected number of DESI sources within 1 arcsec radius 0.014049799119265424
 expected number of DESI sources within 5 arcsec radius 0.3512449779816356
 expected number of DESI sources within 10 arcsec radius 1.4049799119265425


In [33]:
pandas_to_fits(dataframe = desi_cat_scaled_imputed, 
                filename = 'desi_lh.fits',
                table_header_name = 'DESI',
                sky_area_deg2 = DESI_SKYAREA)

# pandas_to_fits(dataframe = desi_cat_scaled_imputed_no_nnmag_grz,
#                 filename = 'desi_lh_no_nnmag_grz.fits',
#                 table_header_name = 'DESI',
#                 sky_area_deg2 = DESI_SKYAREA)

In [34]:
desi_cat_scaled_imputed.query("prior_type == 'grzw1w2'")[['nnmag_grz_orig','nnmag_grzw1_orig', 'nnmag_grzw1w2_orig']].corr()

,nnmag_grz_orig,nnmag_grzw1_orig,nnmag_grzw1w2_orig
nnmag_grz_orig,1.000000,0.394419,0.370474
nnmag_grzw1_orig,0.394419,1.000000,0.781065
nnmag_grzw1w2_orig,0.370474,0.781065,1.000000


In [35]:
desi_cat_scaled_imputed.query("prior_type == 'grzw1w2'").iloc[0]

desi_id                  9011_608391_3304
ra                             166.193423
dec                             56.994008
rel_dered_mag_g                     -99.0
rel_dered_mag_r                     -99.0
rel_dered_mag_z                     -99.0
rel_dered_mag_w1                    -99.0
rel_dered_mag_w2                    -99.0
rel_dered_mag_w3                 0.405444
rel_dered_mag_w4                 0.416527
rel_dered_g_r                       -99.0
rel_dered_r_z                       -99.0
rel_dered_g_z                       -99.0
rel_dered_z_w1                      -99.0
rel_dered_r_w2                      -99.0
rel_dered_w1_w2                     -99.0
rel_dered_z_w3                   0.041039
rel_dered_r_w4                  -0.846337
rel_dered_w3_w4                  -0.03879
prior_type                        grzw1w2
nnmag_grzw1w2                    0.363125
nnmag_grzw1w2_orig               0.363125
nnmag_grzw1                         -99.0
nnmag_grzw1_orig                 0

In [1]:
# desi_cat_scaled_imputed_no_nnmag_grz.query("prior_type == 'grzw1w2'").iloc[0]